<a href="https://colab.research.google.com/github/shearere2/vcu_reu/blob/main/XGBOOST_FRAMEWORK_ETHAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
install.packages('xgboost')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [7]:
install.packages('estimatr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Formula’, ‘Rcpp’, ‘RcppEigen’




In [8]:
install.packages("dplyr") # this is going to be for binarizing the predictor columns

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [9]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [10]:
library(xgboost)


Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice




In [11]:
library(estimatr)

# CISGENDER WOMEN SUBGROUP

### Using GPA as primary predictor

In [97]:
set.seed(2024)
data.intake <- read.csv('gender_female.csv')
rownames(data.intake) <- data.intake$record_id

In [98]:
col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

In [99]:
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

In [100]:
n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

In [101]:
data.intake

,gpa_thresh3_0,role_family,role_friend,role_sigother,role_other,has_mother,has_father,stem_notstem,edu.r_count,edu.i_count,⋯,covid_edu,ptsd_score,trauma_sum,mh_scale,ss_friend,ss_family,ss_sigother,ss_community,age,first_gen
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,0,1,1,1,0,1,1,0,1,4,⋯,6,0,13,0.04761905,5.666667,5.50,5.50,4.75,0,0
2,0,1,0,1,0,1,0,0,0,0,⋯,5,15,10,0.80952381,4.666667,5.50,5.50,4.75,29,0
3,0,1,1,1,0,1,1,0,2,2,⋯,1,3,10,0.09523809,5.666667,5.50,5.50,5.50,53,0
4,0,1,1,1,0,1,1,0,1,3,⋯,7,0,7,0.28571429,5.666667,5.50,5.50,5.00,46,0
6,0,1,1,0,0,1,0,0,2,2,⋯,2,8,12,0.23809524,5.333333,3.75,5.50,4.50,0,0
7,0,1,0,0,0,1,1,0,2,4,⋯,2,4,9,0.14285714,5.333333,4.75,5.50,5.00,28,0
8,0,1,1,1,0,1,0,0,1,2,⋯,0,0,12,0.14285714,5.666667,5.50,5.50,3.75,55,0
9,0,1,1,1,0,1,1,0,3,3,⋯,3,15,8,1.04761905,4.666667,5.25,5.50,5.25,0,0
10,0,1,0,1,0,1,0,0,0,3,⋯,8,40,5,0.42857143,5.333333,4.25,4.50,3.50,0,0


In [102]:
params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

In [103]:
fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

[1]	train-error:0.120536	test-error:1.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.040179	test-error:1.000000 
[21]	train-error:0.026786	test-error:1.000000 
[31]	train-error:0.008929	test-error:1.000000 
[41]	train-error:0.004464	test-error:1.000000 
[51]	train-error:0.004464	test-error:1.000000 
[61]	train-error:0.000000	test-error:1.000000 
[71]	train-error:0.000000	test-error:1.000000 
[81]	train-error:0.000000	test-error:1.000000 
[91]	train-error:0.000000	test-error:1.000000 
[101]	train-error:0.000000	test-error:1.000000 
Stopping. Best iteration:
[1]	train-error:0.120536	test-error:1.000000



In [104]:
pred <- predict(fit_xgb, train.intake)

In [105]:
dif <- dim(data.intake)[1] - length(pred)
dif

[1] 3

In [106]:
for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

In [107]:
ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))

In [108]:
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)

In [109]:
summary(model)


Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)     0.8323    0.04594  18.120 7.461e-46   0.7418  0.92287 225
gpa_thresh3_0  -0.1896    0.08903  -2.129 3.433e-02  -0.3650 -0.01412 225

Multiple R-squared:  0.01854 ,	Adjusted R-squared:  0.01418 
F-statistic: 4.533 on 1 and 225 DF,  p-value: 0.03433

Using PTSD score as primary predictor

In [110]:
set.seed(2024)
data.intake <- read.csv('gender_female.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.071429	test-error:0.666667 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.053571	test-error:0.000000 
[21]	train-error:0.044643	test-error:0.000000 
[31]	train-error:0.022321	test-error:0.333333 
[41]	train-error:0.013393	test-error:0.000000 
[51]	train-error:0.004464	test-error:0.000000 
[61]	train-error:0.004464	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[5]	train-error:0.049107	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.5297    0.03231   16.39 2.875e-40    0.466   0.5934 225
ptsd_score    0.8487    0.07343   11.56 1.457e-24    0.704   0.9934 225

Multiple R-squared:  0.4369 ,	Adjusted R-squared:  0.4344 
F-statistic: 133.6 on 1 and 225 DF,  p-value: < 2.2e-16

Using age as primary predictor

In [111]:
median(data.intake$age)

[1] 20

In [112]:
set.seed(2024)
data.intake <- read.csv('gender_female.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(age = ifelse(age < 21, 1, 0))

col_names <- colnames(data.intake)
if ("age" %in% col_names){
  new_col_order <- c("age", col_names[col_names != "age"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$age==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ age, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.165179	test-error:0.666667 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.031250	test-error:0.333333 
[21]	train-error:0.004464	test-error:0.333333 
[31]	train-error:0.000000	test-error:0.333333 
[41]	train-error:0.000000	test-error:0.666667 
[51]	train-error:0.000000	test-error:0.666667 
[61]	train-error:0.000000	test-error:0.666667 
[71]	train-error:0.000000	test-error:1.000000 
[81]	train-error:0.000000	test-error:1.000000 
[91]	train-error:0.000000	test-error:1.000000 
[101]	train-error:0.000000	test-error:1.000000 
Stopping. Best iteration:
[4]	train-error:0.120536	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ age, data = data.intake, weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)  0.81154    0.06053 13.4078 1.618e-30   0.6923   0.9308 225
age         -0.03458    0.08107 -0.4265 6.701e-01  -0.1943   0.1252 225

Multiple R-squared:  0.0008267 ,	Adjusted R-squared:  -0.003614 
F-statistic: 0.1819 on 1 and 225 DF,  p-value: 0.6701

Using edu.i_count as predidctor

In [113]:
set.seed(2024)
data.intake <- read.csv('gender_female.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.075893	test-error:0.333333 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.044643	test-error:0.333333 
[21]	train-error:0.031250	test-error:0.333333 
[31]	train-error:0.022321	test-error:0.333333 
[41]	train-error:0.013393	test-error:0.333333 
[51]	train-error:0.004464	test-error:0.333333 
[61]	train-error:0.004464	test-error:0.333333 
[71]	train-error:0.004464	test-error:0.333333 
[81]	train-error:0.004464	test-error:0.333333 
[91]	train-error:0.004464	test-error:0.333333 
[101]	train-error:0.000000	test-error:0.333333 
Stopping. Best iteration:
[1]	train-error:0.075893	test-error:0.333333




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.7062    0.05987  11.796 2.558e-25  0.58825   0.8242 225
edu.i_count   0.1270    0.07917   1.604 1.102e-01 -0.02906   0.2830 225

Multiple R-squared:  0.01061 ,	Adjusted R-squared:  0.006208 
F-statistic: 2.571 on 1 and 225 DF,  p-value: 0.1102

# Cisgender Man Subgroup

GPA as primary predictor

In [114]:
set.seed(2024)
data.intake <- read.csv('gender_male.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.188679	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.094340	test-error:0.000000 
[21]	train-error:0.075472	test-error:0.000000 
[31]	train-error:0.037736	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.188679	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)    0.77227     0.1021   7.560 6.339e-10   0.5673   0.9772 52
gpa_thresh3_0  0.07611     0.2213   0.344 7.323e-01  -0.3679   0.5201 52

Multiple R-squared:  0.002269 ,	Adjusted R-squared:  -0.01692 
F-statistic: 0.1183 on 1 and 52 DF,  p-value: 0.7323

PTSD

In [115]:
set.seed(2024)
data.intake <- read.csv('gender_male.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.075472	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.056604	test-error:0.000000 
[21]	train-error:0.056604	test-error:0.000000 
[31]	train-error:0.056604	test-error:0.000000 
[41]	train-error:0.056604	test-error:0.000000 
[51]	train-error:0.037736	test-error:0.000000 
[61]	train-error:0.037736	test-error:0.000000 
[71]	train-error:0.018868	test-error:0.000000 
[81]	train-error:0.018868	test-error:0.000000 
[91]	train-error:0.018868	test-error:0.000000 
[101]	train-error:0.018868	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.075472	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.4615    0.05696   8.103 8.746e-11   0.3472   0.5758 52
ptsd_score    1.1782    0.13625   8.647 1.225e-11   0.9048   1.4516 52

Multiple R-squared:  0.6542 ,	Adjusted R-squared:  0.6476 
F-statistic: 74.78 on 1 and 52 DF,  p-value: 1.225e-11

Age

In [116]:
set.seed(2024)
data.intake <- read.csv('gender_male.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(age = ifelse(age < 21, 1, 0))

col_names <- colnames(data.intake)
if ("age" %in% col_names){
  new_col_order <- c("age", col_names[col_names != "age"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$age==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ age, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.188679	test-error:1.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.037736	test-error:1.000000 
[21]	train-error:0.018868	test-error:1.000000 
[31]	train-error:0.018868	test-error:1.000000 
[41]	train-error:0.000000	test-error:1.000000 
[51]	train-error:0.018868	test-error:1.000000 
[61]	train-error:0.018868	test-error:1.000000 
[71]	train-error:0.000000	test-error:1.000000 
[81]	train-error:0.000000	test-error:1.000000 
[91]	train-error:0.000000	test-error:1.000000 
[101]	train-error:0.000000	test-error:0.000000 
[111]	train-error:0.000000	test-error:0.000000 
[121]	train-error:0.000000	test-error:1.000000 
[131]	train-error:0.000000	test-error:1.000000 
[141]	train-error:0.000000	test-error:1.000000 
[151]	train-error:0.000000	test-error:1.000000 
[161]	train-error:0.000000	test-error:1.000000 
[171]	train-error:0.000000	test-error:0.000000 
[181]	train-


Call:
lm_robust(formula = mh_scale ~ age, data = data.intake, weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.8468     0.2194  3.8597 0.0003154   0.4065   1.2870 52
age          -0.0899     0.2610 -0.3445 0.7318755  -0.6136   0.4338 52

Multiple R-squared:  0.004607 ,	Adjusted R-squared:  -0.01454 
F-statistic: 0.1187 on 1 and 52 DF,  p-value: 0.7319

In [117]:
set.seed(2024)
data.intake <- read.csv('gender_male.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.113208	test-error:1.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.056604	test-error:1.000000 
[21]	train-error:0.037736	test-error:1.000000 
[31]	train-error:0.018868	test-error:1.000000 
[41]	train-error:0.018868	test-error:1.000000 
[51]	train-error:0.000000	test-error:1.000000 
[61]	train-error:0.000000	test-error:1.000000 
[71]	train-error:0.000000	test-error:1.000000 
[81]	train-error:0.000000	test-error:1.000000 
[91]	train-error:0.000000	test-error:1.000000 
[101]	train-error:0.000000	test-error:1.000000 
Stopping. Best iteration:
[1]	train-error:0.113208	test-error:1.000000




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.5402     0.1106   4.883 1.035e-05  0.31820   0.7622 52
edu.i_count   0.3989     0.1652   2.414 1.932e-02  0.06735   0.7305 52

Multiple R-squared:  0.08883 ,	Adjusted R-squared:  0.0713 
F-statistic: 5.829 on 1 and 52 DF,  p-value: 0.01932

# 18-20 subgroup

GPA

In [118]:
set.seed(2024)
data.intake <- read.csv('ages18_20.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.072727	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.090909	test-error:0.000000 
[21]	train-error:0.072727	test-error:0.000000 
[31]	train-error:0.072727	test-error:0.000000 
[41]	train-error:0.054545	test-error:0.000000 
[51]	train-error:0.036364	test-error:0.000000 
[61]	train-error:0.036364	test-error:0.000000 
[71]	train-error:0.036364	test-error:0.000000 
[81]	train-error:0.018182	test-error:0.000000 
[91]	train-error:0.018182	test-error:0.000000 
[101]	train-error:0.018182	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.072727	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)     0.8894    0.09546   9.317 7.854e-13   0.6980   1.0808 54
gpa_thresh3_0  -0.2728    0.24928  -1.095 2.786e-01  -0.7726   0.2269 54

Multiple R-squared:  0.02219 ,	Adjusted R-squared:  0.004079 
F-statistic: 1.198 on 1 and 54 DF,  p-value: 0.2786

PTSD

In [119]:
set.seed(2024)
data.intake <- read.csv('ages18_20.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.127273	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.090909	test-error:0.000000 
[21]	train-error:0.072727	test-error:0.000000 
[31]	train-error:0.054545	test-error:0.000000 
[41]	train-error:0.018182	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.127273	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.4649    0.06553   7.095 2.886e-09   0.3335   0.5963 54
ptsd_score    0.9857    0.13515   7.294 1.373e-09   0.7148   1.2567 54

Multiple R-squared:  0.5403 ,	Adjusted R-squared:  0.5318 
F-statistic:  53.2 on 1 and 54 DF,  p-value: 1.373e-09

Max.emohelp

In [120]:
(data.intake$max.emohelp)

[1]  9 10  7  8  8 10 10  9 10  9  8 10 10 10  9  9 10  9  7 10  8  5 10 10  9
[26]  9 10 10  7 10 10  9 10  8 10  5 10 10 10 10 10  8  9 10  9  9  9 10  7 10
[51] 10  0 10 10  8  0

In [121]:
set.seed(2024)
data.intake <- read.csv('ages18_20.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(max.emohelp = ifelse(max.emohelp >= 9, 1, 0))

col_names <- colnames(data.intake)
if ("max.emohelp" %in% col_names){
  new_col_order <- c("max.emohelp", col_names[col_names != "max.emohelp"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$max.emohelp==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ max.emohelp, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.036364	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.036364	test-error:0.000000 
[21]	train-error:0.036364	test-error:0.000000 
[31]	train-error:0.036364	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.036364	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ max.emohelp, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   1.0020     0.1637   6.120 1.095e-07   0.6737   1.3303 54
max.emohelp  -0.2014     0.1950  -1.033 3.064e-01  -0.5924   0.1896 54

Multiple R-squared:  0.01882 ,	Adjusted R-squared:  0.0006496 
F-statistic: 1.066 on 1 and 54 DF,  p-value: 0.3064

#Ages 21-24

GPA

In [122]:
set.seed(2024)
data.intake <- read.csv('ages21_24.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.208791	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.043956	test-error:0.000000 
[21]	train-error:0.032967	test-error:0.000000 
[31]	train-error:0.000000	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.208791	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)     0.9550    0.08725  10.946 3.169e-18   0.7816   1.1283 90
gpa_thresh3_0  -0.3766    0.12949  -2.909 4.571e-03  -0.6339  -0.1194 90

Multiple R-squared:  0.07475 ,	Adjusted R-squared:  0.06447 
F-statistic: 8.461 on 1 and 90 DF,  p-value: 0.004571

ptsd

In [123]:
set.seed(2024)
data.intake <- read.csv('ages21_24.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.109890	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.065934	test-error:0.000000 
[21]	train-error:0.032967	test-error:0.000000 
[31]	train-error:0.010989	test-error:0.000000 
[41]	train-error:0.010989	test-error:0.000000 
[51]	train-error:0.010989	test-error:0.000000 
[61]	train-error:0.010989	test-error:0.000000 
[71]	train-error:0.010989	test-error:0.000000 
[81]	train-error:0.010989	test-error:0.000000 
[91]	train-error:0.010989	test-error:0.000000 
[101]	train-error:0.010989	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.109890	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)    0.485    0.05096   9.517 2.883e-15   0.3838   0.5863 90
ptsd_score     1.028    0.10830   9.488 3.324e-15   0.8124   1.2427 90

Multiple R-squared:  0.5498 ,	Adjusted R-squared:  0.5448 
F-statistic: 90.01 on 1 and 90 DF,  p-value: 3.324e-15

max.emohelp

In [124]:
(data.intake$max.emohelp)

[1] 10  7  8 10  9 10  8 10  9 10  9  9  0  8 10  9  8 10  9  8  9 10  8  8 10
[26] 10  8 10 10  7 10  9  7 10 10 10 10 10 10 10  8 10 10  8 10 10  8 10 10  9
[51]  7 10  8 10  8 10 10 10 10 10 10  9  0  9  5  8 10  9  7 10  8  0 10 10 10
[76]  8 10 10  7 10 10 10  5 10 10 10 10 10 10  9 10  9

In [126]:
set.seed(2024)
data.intake <- read.csv('ages21_24.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(max.emohelp = ifelse(max.emohelp >= 9, 1, 0))

col_names <- colnames(data.intake)
if ("max.emohelp" %in% col_names){
  new_col_order <- c("max.emohelp", col_names[col_names != "max.emohelp"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$max.emohelp==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ max.emohelp, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.032967	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.032967	test-error:0.000000 
[21]	train-error:0.010989	test-error:0.000000 
[31]	train-error:0.010989	test-error:0.000000 
[41]	train-error:0.010989	test-error:0.000000 
[51]	train-error:0.010989	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.032967	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ max.emohelp, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.6529     0.0824   7.924 5.848e-12 0.489242   0.8167 90
max.emohelp   0.2503     0.1217   2.056 4.263e-02 0.008496   0.4921 90

Multiple R-squared:  0.03105 ,	Adjusted R-squared:  0.02029 
F-statistic: 4.229 on 1 and 90 DF,  p-value: 0.04263

#25+

GPA

In [127]:
set.seed(2024)
data.intake <- read.csv('ages25_63.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.114286	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.085714	test-error:0.000000 
[21]	train-error:0.085714	test-error:0.000000 
[31]	train-error:0.085714	test-error:0.000000 
[41]	train-error:0.085714	test-error:0.000000 
[51]	train-error:0.085714	test-error:0.000000 
[61]	train-error:0.085714	test-error:0.000000 
[71]	train-error:0.028571	test-error:0.000000 
[81]	train-error:0.028571	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.114286	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)     0.6724    0.08301   8.100 1.911e-09   0.5037   0.8411 34
gpa_thresh3_0   0.3946    0.24495   1.611 1.164e-01  -0.1032   0.8924 34

Multiple R-squared:  0.1078 ,	Adjusted R-squared:  0.08153 
F-statistic: 2.595 on 1 and 34 DF,  p-value: 0.1164

PTSD

In [128]:
set.seed(2024)
data.intake <- read.csv('ages25_63.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.085714	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.085714	test-error:0.000000 
[21]	train-error:0.057143	test-error:0.000000 
[31]	train-error:0.028571	test-error:0.000000 
[41]	train-error:0.028571	test-error:0.000000 
[51]	train-error:0.028571	test-error:0.000000 
[61]	train-error:0.028571	test-error:0.000000 
[71]	train-error:0.028571	test-error:0.000000 
[81]	train-error:0.028571	test-error:0.000000 
[91]	train-error:0.028571	test-error:0.000000 
[101]	train-error:0.028571	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.085714	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.6747    0.08454   7.981 2.671e-09  0.50293   0.8465 34
ptsd_score    0.5876    0.24014   2.447 1.972e-02  0.09962   1.0757 34

Multiple R-squared:  0.1674 ,	Adjusted R-squared:  0.1429 
F-statistic: 5.988 on 1 and 34 DF,  p-value: 0.01972

Max.emohelp

In [129]:
(data.intake$max.emohelp)

[1] 10 10 10 10 10  9  9 10  9 10 10 10  9 10  1 10 10 10  9 10  9 10  9  7  9
[26] 10 10 10 10 10  7 10 10 10  8 10

In [130]:
set.seed(2024)
data.intake <- read.csv('ages25_63.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(max.emohelp = ifelse(max.emohelp >= 9, 1, 0))

col_names <- colnames(data.intake)
if ("max.emohelp" %in% col_names){
  new_col_order <- c("max.emohelp", col_names[col_names != "max.emohelp"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$max.emohelp==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ max.emohelp, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.057143	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.000000	test-error:0.000000 
[21]	train-error:0.000000	test-error:0.000000 
[31]	train-error:0.000000	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.057143	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ max.emohelp, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   1.0680     0.2316   4.611 5.463e-05   0.5972   1.5387 34
max.emohelp  -0.3483     0.2485  -1.402 1.701e-01  -0.8534   0.1567 34

Multiple R-squared:  0.04978 ,	Adjusted R-squared:  0.02183 
F-statistic: 1.964 on 1 and 34 DF,  p-value: 0.1701

# First Generation students

GPA

In [132]:
set.seed(2024)
data.intake <- read.csv('firstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.177778	test-error:1.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.044444	test-error:0.000000 
[21]	train-error:0.022222	test-error:0.000000 
[31]	train-error:0.022222	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[5]	train-error:0.044444	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)     0.7739     0.1106   6.997 1.153e-08   0.5510   0.9969 44
gpa_thresh3_0  -0.2311     0.1852  -1.248 2.185e-01  -0.6043   0.1420 44

Multiple R-squared:  0.03385 ,	Adjusted R-squared:  0.01189 
F-statistic: 1.558 on 1 and 44 DF,  p-value: 0.2185

PTSD

In [134]:
set.seed(2024)
data.intake <- read.csv('firstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.066667	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.066667	test-error:0.000000 
[21]	train-error:0.066667	test-error:0.000000 
[31]	train-error:0.044444	test-error:0.000000 
[41]	train-error:0.044444	test-error:0.000000 
[51]	train-error:0.022222	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.066667	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.4256    0.05823   7.309 4.033e-09   0.3083    0.543 44
ptsd_score    1.0405    0.16964   6.133 2.150e-07   0.6986    1.382 44

Multiple R-squared:  0.5757 ,	Adjusted R-squared:  0.566 
F-statistic: 37.62 on 1 and 44 DF,  p-value: 2.15e-07

has_i.emo.so

In [135]:
set.seed(2024)
data.intake <- read.csv('firstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("has_i.emo.so" %in% col_names){
  new_col_order <- c("has_i.emo.so", col_names[col_names != "has_i.emo.so"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$has_i.emo.so==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ has_i.emo.so, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.000000	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.000000	test-error:0.000000 
[21]	train-error:0.000000	test-error:0.000000 
[31]	train-error:0.000000	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.000000	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ has_i.emo.so, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
             Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.66615     0.1280  5.2033 4.914e-06   0.4081   0.9242 44
has_i.emo.so  0.06527     0.1828  0.3571 7.227e-01  -0.3031   0.4336 44

Multiple R-squared:  0.002895 ,	Adjusted R-squared:  -0.01977 
F-statistic: 0.1275 on 1 and 44 DF,  p-value: 0.7227

edu_i.count

In [140]:
set.seed(2024)
data.intake <- read.csv('firstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.111111	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.022222	test-error:0.000000 
[21]	train-error:0.022222	test-error:0.000000 
[31]	train-error:0.022222	test-error:0.000000 
[41]	train-error:0.022222	test-error:0.000000 
[51]	train-error:0.022222	test-error:0.000000 
[61]	train-error:0.022222	test-error:0.000000 
[71]	train-error:0.022222	test-error:0.000000 
[81]	train-error:0.022222	test-error:0.000000 
[91]	train-error:0.022222	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.111111	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.5000     0.1401   3.570 0.0008776  0.21776   0.7823 44
edu.i_count   0.2751     0.1788   1.538 0.1311812 -0.08535   0.6355 44

Multiple R-squared:  0.04298 ,	Adjusted R-squared:  0.02123 
F-statistic: 2.366 on 1 and 44 DF,  p-value: 0.1312

#Non-First Generation

GPA

In [136]:
set.seed(2024)
data.intake <- read.csv('nonfirstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.117886	test-error:0.333333 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.085366	test-error:0.333333 
[21]	train-error:0.036585	test-error:0.333333 
[31]	train-error:0.008130	test-error:0.333333 
[41]	train-error:0.004065	test-error:0.333333 
[51]	train-error:0.000000	test-error:0.333333 
[61]	train-error:0.000000	test-error:0.333333 
[71]	train-error:0.000000	test-error:0.333333 
[81]	train-error:0.000000	test-error:0.333333 
[91]	train-error:0.000000	test-error:0.333333 
[101]	train-error:0.000000	test-error:0.333333 
Stopping. Best iteration:
[6]	train-error:0.097561	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)     0.8251    0.04469  18.464 2.038e-48   0.7371  0.91317 247
gpa_thresh3_0  -0.1101    0.08932  -1.233 2.189e-01  -0.2860  0.06582 247

Multiple R-squared:  0.006517 ,	Adjusted R-squared:  0.002495 
F-statistic:  1.52 on 1 and 247 DF,  p-value: 0.2189

In [137]:
set.seed(2024)
data.intake <- read.csv('nonfirstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.101626	test-error:0.333333 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.052846	test-error:0.000000 
[21]	train-error:0.020325	test-error:0.000000 
[31]	train-error:0.016260	test-error:0.000000 
[41]	train-error:0.016260	test-error:0.000000 
[51]	train-error:0.016260	test-error:0.000000 
[61]	train-error:0.004065	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[2]	train-error:0.073171	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.5291    0.03148   16.81 8.855e-43   0.4671   0.5911 247
ptsd_score    0.8700    0.06923   12.57 2.495e-28   0.7336   1.0063 247

Multiple R-squared:  0.4453 ,	Adjusted R-squared:  0.4431 
F-statistic: 157.9 on 1 and 247 DF,  p-value: < 2.2e-16

has_i.emo.so

In [138]:
set.seed(2024)
data.intake <- read.csv('nonfirstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("has_i.emo.so" %in% col_names){
  new_col_order <- c("has_i.emo.so", col_names[col_names != "has_i.emo.so"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$has_i.emo.so==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ has_i.emo.so, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.004065	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.000000	test-error:0.000000 
[21]	train-error:0.004065	test-error:0.000000 
[31]	train-error:0.004065	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.004065	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ has_i.emo.so, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
             Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.81709    0.04824 16.9383 3.125e-43   0.7221   0.9121 247
has_i.emo.so -0.04129    0.07812 -0.5285 5.976e-01  -0.1952   0.1126 247

Multiple R-squared:  0.001158 ,	Adjusted R-squared:  -0.002886 
F-statistic: 0.2793 on 1 and 247 DF,  p-value: 0.5976

edu.i_count

In [139]:
set.seed(2024)
data.intake <- read.csv('nonfirstgen.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.081301	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.024390	test-error:0.000000 
[21]	train-error:0.020325	test-error:0.000000 
[31]	train-error:0.012195	test-error:0.000000 
[41]	train-error:0.004065	test-error:0.000000 
[51]	train-error:0.004065	test-error:0.000000 
[61]	train-error:0.004065	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.081301	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|)  CI Lower CI Upper  DF
(Intercept)   0.7087    0.05596  12.664 1.175e-28  0.598449   0.8189 247
edu.i_count   0.1454    0.07578   1.918 5.621e-02 -0.003881   0.2946 247

Multiple R-squared:  0.01386 ,	Adjusted R-squared:  0.009871 
F-statistic:  3.68 on 1 and 247 DF,  p-value: 0.05621

# WHITE

GPA

In [145]:
set.seed(2024)
data.intake <- read.csv('white.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.136000	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.088000	test-error:0.000000 
[21]	train-error:0.072000	test-error:0.000000 
[31]	train-error:0.032000	test-error:0.000000 
[41]	train-error:0.024000	test-error:0.000000 
[51]	train-error:0.016000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.136000	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)     0.7263    0.05539  13.111 3.066e-25   0.6166  0.83590 125
gpa_thresh3_0  -0.1679    0.11962  -1.404 1.629e-01  -0.4046  0.06883 125

Multiple R-squared:  0.01252 ,	Adjusted R-squared:  0.004617 
F-statistic:  1.97 on 1 and 125 DF,  p-value: 0.1629

PTSD

In [146]:
set.seed(2024)
data.intake <- read.csv('white.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.088000	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.072000	test-error:0.000000 
[21]	train-error:0.056000	test-error:0.000000 
[31]	train-error:0.056000	test-error:0.000000 
[41]	train-error:0.040000	test-error:0.000000 
[51]	train-error:0.032000	test-error:0.000000 
[61]	train-error:0.024000	test-error:0.000000 
[71]	train-error:0.016000	test-error:0.000000 
[81]	train-error:0.008000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.088000	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.4809    0.03452  13.929 3.363e-27   0.4126   0.5492 125
ptsd_score    0.9614    0.10196   9.429 2.859e-16   0.7596   1.1632 125

Multiple R-squared:  0.5267 ,	Adjusted R-squared:  0.5229 
F-statistic: 88.91 on 1 and 125 DF,  p-value: 2.859e-16

Age

In [147]:
set.seed(2024)
data.intake <- read.csv('white.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(age = ifelse(age < 21, 1, 0))

col_names <- colnames(data.intake)
if ("age" %in% col_names){
  new_col_order <- c("age", col_names[col_names != "age"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$age==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ age, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.168000	test-error:0.500000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.064000	test-error:0.500000 
[21]	train-error:0.008000	test-error:0.500000 
[31]	train-error:0.008000	test-error:0.500000 
[41]	train-error:0.000000	test-error:0.500000 
[51]	train-error:0.000000	test-error:0.500000 
[61]	train-error:0.000000	test-error:0.500000 
[71]	train-error:0.000000	test-error:0.500000 
[81]	train-error:0.000000	test-error:0.500000 
[91]	train-error:0.000000	test-error:0.500000 
[101]	train-error:0.000000	test-error:0.500000 
Stopping. Best iteration:
[3]	train-error:0.120000	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ age, data = data.intake, weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)  0.74847    0.07434 10.0677 8.036e-18   0.6013   0.8956 125
age         -0.07789    0.10113 -0.7702 4.426e-01  -0.2780   0.1223 125

Multiple R-squared:  0.004728 ,	Adjusted R-squared:  -0.003234 
F-statistic: 0.5932 on 1 and 125 DF,  p-value: 0.4426

edu_i.count

In [148]:
set.seed(2024)
data.intake <- read.csv('white.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.072000	test-error:0.500000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.032000	test-error:0.500000 
[21]	train-error:0.016000	test-error:0.500000 
[31]	train-error:0.016000	test-error:0.500000 
[41]	train-error:0.016000	test-error:0.500000 
[51]	train-error:0.016000	test-error:0.500000 
[61]	train-error:0.008000	test-error:0.500000 
[71]	train-error:0.008000	test-error:0.500000 
[81]	train-error:0.000000	test-error:0.500000 
[91]	train-error:0.000000	test-error:0.500000 
[101]	train-error:0.000000	test-error:0.500000 
Stopping. Best iteration:
[1]	train-error:0.072000	test-error:0.500000




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.5813    0.07643   7.605 5.944e-12  0.42998   0.7325 125
edu.i_count   0.1820    0.09964   1.827 7.011e-02 -0.01517   0.3792 125

Multiple R-squared:  0.0245 ,	Adjusted R-squared:  0.01669 
F-statistic: 3.337 on 1 and 125 DF,  p-value: 0.07011

# ASIAN

GPA

In [149]:
set.seed(2024)
data.intake <- read.csv('asian.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.178571	test-error:1.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.080357	test-error:1.000000 
[21]	train-error:0.026786	test-error:0.500000 
[31]	train-error:0.008929	test-error:0.500000 
[41]	train-error:0.000000	test-error:0.500000 
[51]	train-error:0.000000	test-error:0.500000 
[61]	train-error:0.000000	test-error:0.500000 
[71]	train-error:0.000000	test-error:0.500000 
[81]	train-error:0.000000	test-error:0.500000 
[91]	train-error:0.000000	test-error:0.500000 
[101]	train-error:0.000000	test-error:0.500000 
[111]	train-error:0.000000	test-error:0.500000 
Stopping. Best iteration:
[13]	train-error:0.044643	test-error:0.500000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)     0.9326    0.07315  12.750 1.514e-23   0.7877 1.077568 112
gpa_thresh3_0  -0.2296    0.11797  -1.946 5.418e-02  -0.4633 0.004193 112

Multiple R-squared:  0.03446 ,	Adjusted R-squared:  0.02584 
F-statistic: 3.786 on 1 and 112 DF,  p-value: 0.05418

PTSD

In [150]:
set.seed(2024)
data.intake <- read.csv('asian.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.107143	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.062500	test-error:0.000000 
[21]	train-error:0.044643	test-error:0.000000 
[31]	train-error:0.026786	test-error:0.000000 
[41]	train-error:0.026786	test-error:0.000000 
[51]	train-error:0.026786	test-error:0.000000 
[61]	train-error:0.008929	test-error:0.000000 
[71]	train-error:0.008929	test-error:0.000000 
[81]	train-error:0.008929	test-error:0.000000 
[91]	train-error:0.008929	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.107143	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.5554    0.05267  10.546 1.786e-18   0.4511   0.6598 112
ptsd_score    0.8038    0.10276   7.822 3.134e-12   0.6002   1.0074 112

Multiple R-squared:  0.3836 ,	Adjusted R-squared:  0.3781 
F-statistic: 61.18 on 1 and 112 DF,  p-value: 3.134e-12

Age

In [151]:
set.seed(2024)
data.intake <- read.csv('asian.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(age = ifelse(age < 21, 1, 0))

col_names <- colnames(data.intake)
if ("age" %in% col_names){
  new_col_order <- c("age", col_names[col_names != "age"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$age==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ age, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.187500	test-error:0.500000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.071429	test-error:0.000000 
[21]	train-error:0.035714	test-error:0.500000 
[31]	train-error:0.008929	test-error:0.500000 
[41]	train-error:0.000000	test-error:0.500000 
[51]	train-error:0.000000	test-error:0.500000 
[61]	train-error:0.000000	test-error:0.500000 
[71]	train-error:0.000000	test-error:0.500000 
[81]	train-error:0.000000	test-error:0.500000 
[91]	train-error:0.000000	test-error:0.500000 
[101]	train-error:0.000000	test-error:0.500000 
Stopping. Best iteration:
[2]	train-error:0.142857	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ age, data = data.intake, weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)  0.90589     0.0900 10.0652 2.321e-17   0.7276   1.0842 112
age         -0.09956     0.1193 -0.8349 4.056e-01  -0.3358   0.1367 112

Multiple R-squared:  0.006271 ,	Adjusted R-squared:  -0.002602 
F-statistic: 0.697 on 1 and 112 DF,  p-value: 0.4056

edu_i.count

In [152]:
set.seed(2024)
data.intake <- read.csv('asian.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.142857	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.035714	test-error:0.000000 
[21]	train-error:0.026786	test-error:0.000000 
[31]	train-error:0.017857	test-error:0.000000 
[41]	train-error:0.008929	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.142857	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper  DF
(Intercept)   0.7573    0.08923   8.488 9.895e-14  0.58055   0.9341 112
edu.i_count   0.1514    0.11822   1.281 2.030e-01 -0.08285   0.3856 112

Multiple R-squared:  0.0138 ,	Adjusted R-squared:  0.005 
F-statistic:  1.64 on 1 and 112 DF,  p-value: 0.203

# RACE OTHER

GPA

In [153]:
set.seed(2024)
data.intake <- read.csv('race_other.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL

col_names <- colnames(data.intake)
if ("gpa_thresh3_0" %in% col_names){
  new_col_order <- c("gpa_thresh3_0", col_names[col_names != "gpa_thresh3_0"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$gpa_thresh3_0==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ gpa_thresh3_0, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.150943	test-error:1.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.150943	test-error:1.000000 
[21]	train-error:0.113208	test-error:0.000000 
[31]	train-error:0.094340	test-error:0.000000 
[41]	train-error:0.037736	test-error:0.000000 
[51]	train-error:0.018868	test-error:0.000000 
[61]	train-error:0.018868	test-error:0.000000 
[71]	train-error:0.018868	test-error:0.000000 
[81]	train-error:0.000000	test-error:1.000000 
[91]	train-error:0.000000	test-error:1.000000 
[101]	train-error:0.000000	test-error:1.000000 
Stopping. Best iteration:
[7]	train-error:0.132075	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ gpa_thresh3_0, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)    0.82820    0.09776  8.4715 2.307e-11   0.6320    1.024 52
gpa_thresh3_0  0.05455    0.25288  0.2157 8.301e-01  -0.4529    0.562 52

Multiple R-squared:  0.001247 ,	Adjusted R-squared:  -0.01796 
F-statistic: 0.04652 on 1 and 52 DF,  p-value: 0.8301

PTSD

In [154]:
set.seed(2024)
data.intake <- read.csv('race_other.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(ptsd_score = ifelse(ptsd_score >= 33, 1, 0))

col_names <- colnames(data.intake)
if ("ptsd_score" %in% col_names){
  new_col_order <- c("ptsd_score", col_names[col_names != "ptsd_score"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$ptsd_score==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ ptsd_score, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.094340	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.018868	test-error:1.000000 
[21]	train-error:0.018868	test-error:1.000000 
[31]	train-error:0.000000	test-error:1.000000 
[41]	train-error:0.000000	test-error:1.000000 
[51]	train-error:0.000000	test-error:1.000000 
[61]	train-error:0.000000	test-error:1.000000 
[71]	train-error:0.000000	test-error:1.000000 
[81]	train-error:0.000000	test-error:1.000000 
[91]	train-error:0.000000	test-error:1.000000 
[101]	train-error:0.000000	test-error:1.000000 
Stopping. Best iteration:
[1]	train-error:0.094340	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ ptsd_score, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.5190     0.0755   6.874 7.874e-09   0.3675   0.6705 52
ptsd_score    0.9432     0.1372   6.875 7.842e-09   0.6679   1.2185 52

Multiple R-squared:  0.4914 ,	Adjusted R-squared:  0.4816 
F-statistic: 47.26 on 1 and 52 DF,  p-value: 7.842e-09

Age

In [155]:
set.seed(2024)
data.intake <- read.csv('race_other.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(age = ifelse(age < 21, 1, 0))

col_names <- colnames(data.intake)
if ("age" %in% col_names){
  new_col_order <- c("age", col_names[col_names != "age"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$age==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ age, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.150943	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.037736	test-error:0.000000 
[21]	train-error:0.000000	test-error:0.000000 
[31]	train-error:0.000000	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.150943	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ age, data = data.intake, weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept) 0.828656     0.1575 5.26259 2.731e-06   0.5127   1.1446 52
age         0.007848     0.1871 0.04194 9.667e-01  -0.3677   0.3834 52

Multiple R-squared:  3.707e-05 ,	Adjusted R-squared:  -0.01919 
F-statistic: 0.001759 on 1 and 52 DF,  p-value: 0.9667

edu_i.count

In [156]:
set.seed(2024)
data.intake <- read.csv('race_other.csv')
rownames(data.intake) <- data.intake$record_id
data.intake$record_id <- NULL
data.intake <- data.intake %>%
  mutate(edu.i_count = ifelse(edu.i_count <= 2, 1, 0))

col_names <- colnames(data.intake)
if ("edu.i_count" %in% col_names){
  new_col_order <- c("edu.i_count", col_names[col_names != "edu.i_count"])
  data.intake <- data.intake[, new_col_order]}

n <- nrow(data.intake)
train_rows <- sample(1:n, 0.99 * n, replace = F)
train.intake <- xgb.DMatrix(data.matrix(data.intake[train_rows, -1]),
                            label = data.intake[train_rows, 1])
test.intake <- xgb.DMatrix(data.matrix(data.intake[-train_rows, -1]),
                            label = data.intake[-train_rows, 1])

params <- list(
  objective = "binary:logistic",
  learning_rate = 0.05,
  subsample = 0.9,
  colsample_bynode = 1,
  max_depth = 10,
  scale_pos_weight = 1
)

fit_xgb <- xgb.train(
  params,
  data = train.intake,
  watchlist = list(train = train.intake, test = test.intake),
  eval.metric = "error",
  early_stopping_rounds = 100,
  print_every_n = 10,
  nrounds = 10000
)

pred <- predict(fit_xgb, train.intake)
dif <- dim(data.intake)[1] - length(pred)

for (x in 1:dif) {
  pred <- c(pred, mean(pred))
}

ipw <- ifelse(data.intake$edu.i_count==1, 1/pred, 1/(1-pred))
model <- lm_robust(mh_scale ~ edu.i_count, data=data.intake, weights=ipw)
summary(model)

[1]	train-error:0.094340	test-error:0.000000 
Multiple eval metrics are present. Will use test_error for early stopping.
Will train until test_error hasn't improved in 100 rounds.

[11]	train-error:0.056604	test-error:0.000000 
[21]	train-error:0.018868	test-error:0.000000 
[31]	train-error:0.018868	test-error:0.000000 
[41]	train-error:0.000000	test-error:0.000000 
[51]	train-error:0.000000	test-error:0.000000 
[61]	train-error:0.000000	test-error:0.000000 
[71]	train-error:0.000000	test-error:0.000000 
[81]	train-error:0.000000	test-error:0.000000 
[91]	train-error:0.000000	test-error:0.000000 
[101]	train-error:0.000000	test-error:0.000000 
Stopping. Best iteration:
[1]	train-error:0.094340	test-error:0.000000




Call:
lm_robust(formula = mh_scale ~ edu.i_count, data = data.intake, 
    weights = ipw)

Weighted, Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper DF
(Intercept)   0.7442     0.1123  6.6247 1.963e-08   0.5188   0.9697 52
edu.i_count   0.1349     0.1672  0.8065 4.236e-01  -0.2007   0.4704 52

Multiple R-squared:  0.0108 ,	Adjusted R-squared:  -0.008221 
F-statistic: 0.6505 on 1 and 52 DF,  p-value: 0.4236